##Hugging Face Transformers for AutoTokenizer and CodeBERT

In [8]:
!pip install transformers

##Tree-sitter for Syntax-aware Tokenization

In [9]:
!pip install tree-sitter-language-pack

##Build solidity parser

In [10]:
from tree_sitter_language_pack import get_language, get_parser

solidity_lang = get_language("solidity")
parser = get_parser("solidity")


##Parsing Solidity code into tokens

In [20]:
import os

def parsing_code(path_to_files):
  dir_path = os.path.dirname(path_to_files)
  for file_name in os.listdir(dir_path):
      file_path = os.path.join(dir_path, file_name)
      if os.path.isfile(file_path):
          with open(file_path, 'r', encoding='utf-8') as f:
              code = f.read()
              code_bytes = bytes(code, 'utf-8')
              tree = parser.parse(code_bytes)
              print(tree)

In [ ]:
parsing_code("/data/owasp2025_top_10_vul/vulnerable/")

In [25]:
code = open("/content/dos.sol").read()
code = bytes(code, 'utf-8')

In [26]:
tree = parser.parse(code)
print(tree)


##Extracting tokens from solidity code

In [27]:
def extract_tokens(node, code):
    tokens = []
    if node.child_count == 0:
        token_text = code[node.start_byte:node.end_byte].decode('utf-8')
        tokens.append((node.type, token_text))
    for child in node.children:
        tokens.extend(extract_tokens(child, code))
    return tokens

tokens = extract_tokens(tree.root_node, code)
for token_type, token_text in tokens:
    print(f"{token_type}: {token_text}")

pragma: pragma
solidity: solidity
^:  ^
solidity_version: 0.8.24
;: ;
contract: contract
identifier: Solidity_DOS
{: {
address: address
public: public
identifier: king
;: ;
uint256: uint256
public: public
identifier: balance
;: ;
function: function
identifier: claimThrone
(: (
): )
external: external
payable: payable
{: {
identifier: require
(: (
identifier: msg
.: .
identifier: value
>: >
identifier: balance
,: ,
": "
": "
): )
;: ;
(: (
bool: bool
identifier: sent
,: ,
): )
=: =
identifier: king
.: .
identifier: call
{: {
identifier: value
:: :
identifier: balance
}: }
(: (
": "
": "
): )
;: ;
identifier: require
(: (
identifier: sent
,: ,
": "
": "
): )
;: ;
identifier: balance
=: =
identifier: msg
.: .
identifier: value
;: ;
identifier: king
=: =
identifier: msg
.: .
identifier: sender
;: ;
}: }
}: }
